In [1]:
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay
from dateutil.relativedelta import relativedelta
from collections import defaultdict, deque
import pprint

from commons import read_xls, get_stock_data, save_csv, get_holding_quantities
from commons import min_consecutive_sum_kadane, get_curr_prices_from_holdings
from consts import HOLDING_FILE

import os
from consts import OUT_DIR

In [2]:
def remove_company_sold_txns(company_data, qty_id, company):
    q = deque()
    data = company_data.values
    for d in data:
        if d[qty_id] < 0:
            qty_to_remove = -1 * d[qty_id] #make this positive for ease of math
            print(f'{company}: remove already sold qty = {qty_to_remove}')
            while(q and qty_to_remove):
                if q[0][qty_id] <= qty_to_remove: # peek before removing to check for partially sold lot
                    popped = q.popleft()
                    qty_to_remove -= popped[qty_id]
                else:
                    print('# remove partial qty from the qty bought and update qty in-place with qty left')
                    popped = q.popleft()
                    popped[qty_id] = popped[qty_id] - qty_to_remove
                    q.appendleft(popped)
        else:
            q.append(d)

    return list(q)

def remove_sold_txns(df):
    net_txns = list()
    cols = df.columns
    for company, qty in get_holding_quantities().items():
        company_data = df[df['COMPANY'] == company]
        qty = int(qty)    
        qty_id = company_data.columns.get_loc('Qty')
        
        company_txns_in_scope = remove_company_sold_txns(company_data, qty_id, company)
    
        ### Check if the data in Breakdown tab matches, Holdings
        net_qty = 0
        for c in company_txns_in_scope:
            net_qty += c[qty_id]
        # print(f'net_qty = {net_qty} AND qty = {qty}')
        assert(net_qty == qty)
    
        net_txns += company_txns_in_scope
    
    return pd.DataFrame(net_txns, columns=cols)

def append_pnl(df, today=None):
    if not today:
        today = datetime.datetime.today()
        
    start = today - BDay(1) # Get prev business date as market didnt start today
    end = today

#     if datetime.time(today.hour, today.minute, today.second) < datetime.time(9, 15, 5):
#         print('Markets didnt start yet, getting data from prev business dates')
#         end = start
#         start -= BDay(2)
    
    try:
        print(f'****** Calling for stock data between {start} and {end} *******')

        curr_prices = get_stock_data(start=start, end=end, print_data=False)
        num_days = curr_prices.shape[0] - 1 ## (2, 20) since there might be two dates' data pulled in this range of start/end zones - yesterday, and today
                                            ## pick the latest data and rename below as CurrPrice
        curr_prices = curr_prices.reset_index().transpose().reset_index().rename(columns={num_days: 'currPrice'}) #.rename(columns={'index': 'Symbol'})
        curr_prices = curr_prices.rename(columns={'Ticker': 'index'}) 
        curr_prices = curr_prices[['index', 'currPrice']]
    except e: 
        print(e)
        print(f'***** reading current prices from Holdings.txt as yahoo crapped again!!! Prices will be as of {HOLDING_FILE}!!!')
        curr_prices = get_curr_prices_from_holdings()
        
    df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
    df = df.drop('index', axis=1)
    df = df.sort_values(by='Date', ascending=True)
    df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']

#     def flip_sells(t, val):
#         if t == 'sell':
#             val *= -1
            
#         return val

#     df['PnL']      = df.apply(lambda x: flip_sells(x['Trade Type'], x['PnL']), axis=1)
#     df['Quantity'] = df.apply(lambda x: flip_sells(x['Trade Type'], x['Quantity']), axis=1)
    return df

def get_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
#     df = df[df['Trade Date'] > str(date)] # Get trades after this date
    df = df.sort_values(by='Date', ascending=False)
    data = {}
    
    for c, q in get_holding_quantities().items():
        print(f'***************** {c}: {q} ************************')
        df_c = df[df['COMPANY'] == c]
        df_dict = df_c.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
        ltcg, stcg, ltcg_qty, stcg_qty = 0, 0, 0, 0
        data[c] = defaultdict(int)
        data[c]['total_qty'] = 0
        
        for i, (comp, dt, bp, qty, cp, pnl) in enumerate(zip(df_dict['COMPANY'], df_dict['Date'], df_dict['BuyPrice'], \
                                             df_dict['Qty'], df_dict['currPrice'], df_dict['PnL'])):
            i += 1 
            if data[c]['total_qty'] > q: # Dont process older txns' which were squared off earlier
                print(f"Going back to past quantities... break!: {data[c]['total_qty']} > {q}")
                break 
                
            if dt.date() <= stcg_cutoff_date:
                ### LTCG
#                 print(f'LTCG {i} {dt.date()} >= {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                data[c]['ltcg_qty'] += qty
                data[c]['ltcg'] += pnl
            else:
                ### STCG
#                 print(f'STCG {i} {dt.date()} <  {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                
                # STCLoss only if prev last LTCG txn was negative
                if 'stcg_qty' not in data[c]: # init stcg total qty 
                    data[c]['stcl_qty'] = data[c]['ltcg_qty']
                    data[c]['stcg_qty'] = data[c]['ltcg_qty']

                if pnl < 0: ## only if it is a loss
                    data[c]['stcl_qty'] += qty
                    data[c]['stcl'] += pnl
                else:
                    data[c]['stcg_qty'] += qty
                    data[c]['stcg'] += pnl    
            
                
            data[c]['total_qty'] += qty
            data[c]['total'] += pnl
            
    return data

def get_max_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
    data = {}
    
    for c in get_holding_quantities().keys():
        df_c = df[(df['Date'] > str(stcg_cutoff_date)) & (df['COMPANY'] == c)] # Get trades after this date
        df_dict = df_c.to_dict('list')
        data[c] = min_consecutive_sum_kadane(df_dict['PnL'], df_dict['Date'])

    return {k: v for k, v in sorted(data.items(), key=lambda item: item[1])} # sort data by value before returning

def get_max_ltcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Long Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
    data = {}
    
    for c in get_holding_quantities().keys():
        df_c = df[(df['Date'] <= str(stcg_cutoff_date)) & (df['COMPANY'] == c)] # Get trades before this date
        df_dict = df_c.to_dict('list')
        data[c] = min_consecutive_sum_kadane(df_dict['PnL'], df_dict['Date'])

        # Add qty to sell to encash this ltc loss - need to sell all qtys till the end date of the longest loss sequence
        end_date = data[c][-1]
        qty = 0
        if isinstance(end_date, str):
            end_date = datetime.datetime.strptime(end_date, '%d%b%Y')
            
            for d, q in zip(df_dict['Date'], df_dict['Qty']):
                if d <= end_date:
                    qty += q

        data[c].append(qty)

    return {k: v for k, v in sorted(data.items(), key=lambda item: item[1])} # sort data by value before returning


In [5]:
# df = read_xls(HOLDING_FILE, tab=5)
print(f'************************ Calculating as per: {HOLDING_FILE} ***************************\n')
df = pd.ExcelFile(HOLDING_FILE)
# print(xl.sheet_names)  # see all sheet names
df = df.parse('Breakdown') # Read Breakdown sheet

df['COMPANY'] = df['COMPANY'].apply(lambda x: x.strip())
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]

df = append_pnl(df)
df = remove_sold_txns(df)

# save_csv(df, 'ltcg_stcg')

# LTCG loss offset
ltcg = pd.DataFrame(get_max_ltcl(df))
ltcg.index = pd.Index(['MaxLTCGLoss','From','To', 'SellQty'], name='Company')
print(ltcg.T)

print('******* Max STCL - longest STCL loss streak *******')
stcl = pd.DataFrame(get_max_stcl(df))

stcg_ltcg = pd.DataFrame(get_stcl(df)).T.sort_values(by='stcl', ascending=True)

file_name = os.path.join(OUT_DIR, f'ltcg_stcg_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.xlsx')
with pd.ExcelWriter(file_name) as writer: 
    df.sort_values(by='Date', ascending=True).to_excel(writer, sheet_name='All Txns')
    ltcg.T.to_excel(writer, sheet_name='LTCG')
    stcl.T.to_excel(writer, sheet_name='STCG')
    stcg_ltcg.to_excel(writer, sheet_name='LTCG_STCG')
print('Saved: ', file_name)

************************ Calculating as per: ~/Documents/apps/docs/Holdings_18Dec24.xlsx ***************************

****** Calling for stock data between 2024-12-19 01:14:27.595389 and 2024-12-20 01:14:27.595389 *******


[*****                 10%%                      ]  2 of 20 completed

****** Getting stock data between 2024-12-19 01:14:27.595389 and 2024-12-20 01:14:27.595389 *******


[*********************100%%**********************]  20 of 20 completed


****** Got stock data between 2024-12-19 01:14:27.595389 and 2024-12-20 01:14:27.595389 *******
HAPPSTMNDS: remove already sold qty = 359
RAJRATAN: remove already sold qty = 215
PRINCEPIPE: remove already sold qty = 287
Company      MaxLTCGLoss       From         To SellQty
BERGEPAINT -32989.798035  09Dec2021  18Nov2022     161
DMART      -29614.157715  21Oct2021  19Oct2023      79
SONACOMS   -10531.597046  15Nov2021  08Feb2022     121
BAJFINANCE  -9579.195312  23Jun2023  20Dec2023      46
RBA         -7427.450523  21Jan2022  06May2022     245
RAJRATAN         -3976.0  22Nov2023  22Nov2023      16
DEVYANI     -1257.149689  21Aug2023  21Aug2023     281
FINEORG           -120.0  13Oct2023  13Oct2023      41
KPITTECH               0          1          0       0
AFFLE                  0          1          0       0
DEEPAKNTR              0          1          0       0
NAUKRI                 0          1          0       0
DIVISLAB               0          1          0       0
IONEXCHANG

In [ ]:
# with pd.ExcelWriter(os.path.join(OUT_DIR, f'ltcg_stcg_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.xlsx')) as writer: 
#     df.sort_values(by='stcl', ascending=True).to_excel(writer, sheet_name='All Txns')
#     ltcg.T.to_excel(writer, sheet_name='LTCG')
#     stcl.T.to_excel(writer, sheet_name='STCG')
#     stcg_ltcg.to_excel(writer, sheet_name='LTCG_STCG')
stcg_ltcg

In [ ]:
# print(s_df)
# s_df.to_dict()
# sum(s_df['Quantity'])
s_df['PnL'] = (s_df['currPrice'] - s_df['Price']) * s_df['Quantity']
print(s_df)
print(sum(s_df[s_df['Trade Type'] == 'buy']['PnL']) - sum(s_df[s_df['Trade Type'] == 'sell']['PnL']))

def f(t, pnl):
    if t == 'sell':
        return -1*pnl
    return pnl

s_df['PnL'] = s_df.apply(lambda x: f(x['Trade Type'], x['PnL']), axis=1)
s_df
# s_df['Trade Type', 'PnL'].apply(lambda x, y: f(x, y))
# s_df[['Trade Type', 'PnL']]

In [ ]:
datetime.datetime.today().minute
# datetime.datetime.today() - BDay(1)
t = datetime.time(9, 30, 59)
t.hour
# datetime.time(datetime.datetime.today())
datetime.time(datetime.datetime.today().hour, datetime.datetime.today().minute) < datetime.time(9, 15, 5)
datetime.datetime.today().second

today = datetime.datetime.today()
today - BDay(2), today - BDay(1) # Get prev business date as market didnt start today

In [ ]:
curr_prices = get_curr_prices_from_holdings()
# print(curr_prices)
df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
df = df.drop('index', axis=1)
df = df.sort_values(by='Date', ascending=True)
df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']
df